# 00 Data Cleansing

This script reads and cleanses: 
- 01. Actual generation data (From ENTSO-e) orignal .xlsx format 
- 02. CO2 equivalent data (From ElectricityMap's github repository) .json format  


In [1]:
# Importing packages
import pandas as pd
import numpy as np
import xlrd
import os
import sqlite3
import json

## 01. Actual generation data

In [2]:
# Reads file and defines row 4 as header
df =pd.read_excel(os.path.join("ActualGeneration.xlsx"),
     engine='openpyxl', header=4)

# Drop empty rows 
df.dropna(subset = ["MTU"], inplace=True)
                    
# Create date field
df['Date'] = np.where(pd.isnull(df["Biomass"]), df["MTU"],np.nan)

# Replacing na values in date with the date above
df['Date'].fillna( method ='ffill', inplace = True)
df.dropna(subset = ["Biomass"], inplace=True)

# Defines starttime from time interval
df['StartTime']= df['MTU'].str.slice(start=0, stop=5)+":00"

# Combine Date & Startime to get datetime field
df['DateTime']= df['Date'] + ' ' + df['StartTime']

# Save clean version 
df.to_csv('ActualGeneration.csv')

In [22]:
#Initiate a new SQLite database connection object and assign this object to the variable 'dn_conn'
db_conn = sqlite3.connect("EM.db")

In [23]:
# establish a cursor object and assign to c
c = db_conn.cursor()

In [83]:
# Create empty table for actual generation
c.execute(
    """
    CREATE TABLE IF NOT EXISTS ActualGeneration (
        Solar REAL, Biomass REAL );
     """
)

In [84]:
# TODO: Only sample fields for now
df = df[['Solar','Biomass']]
df.head()
# Populate Tables
df.to_sql('ActualGeneration', db_conn, if_exists='append', index=False)

In [85]:
#pd.read_sql("SELECT * FROM ActualGeneration LIMIT 10", db_conn)

## 02. CO2 Equivalent Data

In [103]:
with open('co2eq_parameters.json', 'r') as f:
    data = json.load(f)
df_CO2 = pd.DataFrame(data)



In [104]:
df_CO2.head() 


,emissionFactors,fallbackZoneMixes,isLowCarbon,isRenewable
defaults,{'battery charge': {'_comment': 'Emissions are...,{'_source': 'BP Statistical Review of World En...,"{'battery charge': True, 'battery discharge': ...","{'battery charge': True, 'battery discharge': ..."
zoneOverrides,{'AR': {'hydro discharge': {'source': '2017 av...,"{'AM': {'_source': 'Tomorrow', 'powerOriginRat...","{'GB-ORK': {'unknown': True}, 'UA': {'unknown'...","{'FO': {'unknown': True}, 'GB-ORK': {'unknown'..."


In [111]:
test = df_CO2.iloc[1]['emissionFactors']

print(test)

{'AR': {'hydro discharge': {'source': '2017 average by Tomorrow', 'value': 297.18891097799343}}, 'AT': {'battery discharge': {'source': '2019 average by Tomorrow', 'value': 173.4729702559071}, 'hydro discharge': {'source': '2019 average by Tomorrow', 'value': 173.4729702559071}}, 'AUS-NSW': {'hydro discharge': {'source': '2017 average by Tomorrow', 'value': 726.0064734066499}}, 'AUS-QLD': {'hydro discharge': {'source': '2017 average by Tomorrow', 'value': 766.6875812088138}}, 'AUS-SA': {'battery discharge': {'source': '2019 average by Tomorrow', 'value': 264.9516942784522}, 'hydro discharge': {'source': '2019 average by Tomorrow', 'value': 264.9516942784522}}, 'AUS-TAS': {'hydro discharge': {'source': '2017 average by Tomorrow', 'value': 128.59069995581274}}, 'AUS-VIC': {'hydro discharge': {'source': '2017 average by Tomorrow', 'value': 682.6761763130781}}, 'AUS-WA': {'hydro discharge': {'source': '2017 average by Tomorrow', 'value': 611.2821372774696}}, 'BA': {'hydro discharge': {'sou

In [112]:
dfn = pd.read_json(test)
print(df)

ValueError: Invalid file path or buffer object type: <class 'dict'>